## Uber Eats Restaurant Data

In [1]:
# Dependencies
import pandas as pd
import os 
import csv 
from sqlalchemy import create_engine

#### Import CSV data

In [2]:
csvpath = os.path.join("Resources", "CSVs", "restaurants.csv")

In [3]:
restaurant_data = pd.read_csv(csvpath)

In [4]:
restaurant_data.head()

,id,position,name,score,ratings,category,price_range,full_address,zip_code,lat,lng
0,1,19,PJ Fresh (224 Daniel Payne Drive),NaN,NaN,"Burgers, American, Sandwiches",$,"224 Daniel Payne Drive, Birmingham, AL, 35207",35207,33.562365,-86.830703
1,2,9,J' ti`'z Smoothie-N-Coffee Bar,NaN,NaN,"Coffee and Tea, Breakfast and Brunch, Bubble Tea",NaN,"1521 Pinson Valley Parkway, Birmingham, AL, 35217",35217,33.583640,-86.773330
2,3,6,Philly Fresh Cheesesteaks (541-B Graymont Ave),NaN,NaN,"American, Cheesesteak, Sandwiches, Alcohol",$,"541-B Graymont Ave, Birmingham, AL, 35204",35204,33.509800,-86.854640
3,4,17,Papa Murphy's (1580 Montgomery Highway),NaN,NaN,Pizza,$,"1580 Montgomery Highway, Hoover, AL, 35226",35226,33.404439,-86.806614
4,5,162,Nelson Brothers Cafe (17th St N),4.7,22.0,"Breakfast and Brunch, Burgers, Sandwiches",NaN,"314 17th St N, Birmingham, AL, 35203",35203,33.514730,-86.811700


In [5]:
csvpath = os.path.join("Resources", "CSVs", "restaurant-menus.csv")

In [6]:
menu_data = pd.read_csv(csvpath)

In [7]:
menu_data.head()

,restaurant_id,category,name,description,price
0,1,Extra Large Pizza,Extra Large Meat Lovers,Whole pie.,15.99 USD
1,1,Extra Large Pizza,Extra Large Supreme,Whole pie.,15.99 USD
2,1,Extra Large Pizza,Extra Large Pepperoni,Whole pie.,14.99 USD
3,1,Extra Large Pizza,Extra Large BBQ Chicken &amp; Bacon,Whole Pie,15.99 USD
4,1,Extra Large Pizza,Extra Large 5 Cheese,Whole pie.,14.99 USD


#### Data Clean-up

In [9]:
# original full list of restaurants
original_restaurant_df = restaurant_data.copy()
# number of rows
len(original_restaurant_df.index)

5000

In [10]:
# expand full_address column out to street, city, and state
original_restaurant_df[['street', 'city', 'state', 'zip']] = original_restaurant_df['full_address'].str.split(',', 3, expand=True)

# delete zip and position columns
del original_restaurant_df['zip']

In [11]:
# reorder columns
original_restaurant_df = original_restaurant_df.reindex(columns=['id','name', 'score','ratings','category', 'price_range',
                                              'full_address', 'street', 'city', 'state', 'zip_code', 'lat', 'lng'])

In [12]:
# set index to id
original_restaurant_df.set_index("id", inplace = True)
original_restaurant_df

,name,score,ratings,category,price_range,full_address,street,city,state,zip_code,lat,lng
id,,,,,,,,,,,,
1,PJ Fresh (224 Daniel Payne Drive),NaN,NaN,"Burgers, American, Sandwiches",$,"224 Daniel Payne Drive, Birmingham, AL, 35207",224 Daniel Payne Drive,Birmingham,AL,35207,33.562365,-86.830703
2,J' ti`'z Smoothie-N-Coffee Bar,NaN,NaN,"Coffee and Tea, Breakfast and Brunch, Bubble Tea",NaN,"1521 Pinson Valley Parkway, Birmingham, AL, 35217",1521 Pinson Valley Parkway,Birmingham,AL,35217,33.583640,-86.773330
3,Philly Fresh Cheesesteaks (541-B Graymont Ave),NaN,NaN,"American, Cheesesteak, Sandwiches, Alcohol",$,"541-B Graymont Ave, Birmingham, AL, 35204",541-B Graymont Ave,Birmingham,AL,35204,33.509800,-86.854640
4,Papa Murphy's (1580 Montgomery Highway),NaN,NaN,Pizza,$,"1580 Montgomery Highway, Hoover, AL, 35226",1580 Montgomery Highway,Hoover,AL,35226,33.404439,-86.806614
5,Nelson Brothers Cafe (17th St N),4.7,22.0,"Breakfast and Brunch, Burgers, Sandwiches",NaN,"314 17th St N, Birmingham, AL, 35203",314 17th St N,Birmingham,AL,35203,33.514730,-86.811700
...,...,...,...,...,...,...,...,...,...,...,...,...
4996,Legend Larry's,NaN,NaN,"Wings, American, Fast Food",$$,"921 S 10th St, Manitowoc, WI, 54220",921 S 10th St,Manitowoc,WI,54220,44.089550,-87.660370
4997,Subway (311 N Rapids Rd),NaN,NaN,"Fast Food, Sandwich, American",$,"311 N Rapids Rd, Manitowoc, WI, 54220",311 N Rapids Rd,Manitowoc,WI,54220,44.098060,-87.700210
4998,Walgreens (2702 CALUMET AVE),NaN,NaN,"Pharmacy, Convenience, Everyday Essentials, Baby",$,"2702 Calumet Ave, Manitowoc, WI, 54220",2702 Calumet Ave,Manitowoc,WI,54220,44.087770,-87.681570


In [13]:
restaurant_df = restaurant_data.copy()
restaurant_df.head(10)

,id,position,name,score,ratings,category,price_range,full_address,zip_code,lat,lng
0,1,19,PJ Fresh (224 Daniel Payne Drive),NaN,NaN,"Burgers, American, Sandwiches",$,"224 Daniel Payne Drive, Birmingham, AL, 35207",35207,33.562365,-86.830703
1,2,9,J' ti`'z Smoothie-N-Coffee Bar,NaN,NaN,"Coffee and Tea, Breakfast and Brunch, Bubble Tea",NaN,"1521 Pinson Valley Parkway, Birmingham, AL, 35217",35217,33.583640,-86.773330
2,3,6,Philly Fresh Cheesesteaks (541-B Graymont Ave),NaN,NaN,"American, Cheesesteak, Sandwiches, Alcohol",$,"541-B Graymont Ave, Birmingham, AL, 35204",35204,33.509800,-86.854640
3,4,17,Papa Murphy's (1580 Montgomery Highway),NaN,NaN,Pizza,$,"1580 Montgomery Highway, Hoover, AL, 35226",35226,33.404439,-86.806614
4,5,162,Nelson Brothers Cafe (17th St N),4.7,22.0,"Breakfast and Brunch, Burgers, Sandwiches",NaN,"314 17th St N, Birmingham, AL, 35203",35203,33.514730,-86.811700
5,6,75,Ocean Restaurant,NaN,NaN,"Seafood, Sushi, Steak",$$,"1218 20th Street South, Birmingham, AL, 35205",35205,33.498630,-86.795740
6,7,27,Jinsei Sushi,4.7,63.0,"Sushi, Asian, Japanese",$,"1830 29th Ave S, Birmingham, AL, 35209",35209,33.480440,-86.790440
7,8,26,Little India,NaN,NaN,"Vegetarian, Asian, Asian Fusion, Chinese, Indi...",$,"2236 Highland Ave, Birmingham, AL, 35205",35205,33.502259,-86.790298
8,9,136,Captain D's (1284 Decatur Hwy),NaN,NaN,"Seafood, Fast Food, Fish and Chips, American",$,"1284 Decatur Hwy, Gardendale, AL, 35071",35071,33.657100,-86.811970
9,10,51,Cajun Bistro Express,NaN,NaN,"Desserts, Vegetarian, Coffee and Tea",$,"1515 Pearson Ave SW, Birmingham, AL, 35211",35211,33.486840,-86.856300


In [14]:
# drop position columnn from restaurant_data
restaurant_df = restaurant_data.drop(['position'], axis=1)

# drop rows with NaN values in score and rating (using code to keep rows that are not null)
restaurant_df = restaurant_df[restaurant_df['score'].notna()]
restaurant_df = restaurant_df[restaurant_df['ratings'].notna()]

# drop rows with Nan values in price_range
restaurant_df = restaurant_df.dropna(subset = ['price_range'])
restaurant_df

,id,name,score,ratings,category,price_range,full_address,zip_code,lat,lng
6,7,Jinsei Sushi,4.7,63.0,"Sushi, Asian, Japanese",$,"1830 29th Ave S, Birmingham, AL, 35209",35209,33.480440,-86.790440
13,14,Panera (521 Fieldstown Road),4.6,44.0,"Breakfast and Brunch, salad, Sandwich, Family ...",$,"521 Fieldstown Road, Gardendale, AL, 35071",35071,33.651407,-86.819247
15,16,Jeni's Splendid Ice Cream (Pepper Place),5.0,20.0,"Ice Cream &amp; Frozen Yogurt, Comfort Food, D...",$$$,"219 29th St S, Birmingham, AL, 35233",35233,33.516600,-86.789950
18,19,Falafel Cafe,4.9,48.0,"Middle Eastern, Mediterranean, Vegetarian, Gre...",$,"401 19th St S, Birmingham, AL, 35233",35233,33.508353,-86.803170
19,20,MrBeast Burger (838 Odum Road),3.7,19.0,"American, Burgers, Sandwich",$$,"838 Odum Road, Gardendale, AL, 35071",35071,33.645480,-86.826260
...,...,...,...,...,...,...,...,...,...,...
4951,4952,Toppers Pizza (Menomonee Falls),4.2,49.0,Pizza,$,"N88W15515 Main St., Menomonie Falls, WI, 53051",53051,43.178895,-88.102625
4957,4958,Jersey Mike’s Subs (149 North Mall Drive),4.9,38.0,"Sandwich, American",$,"149 North Mall Drive, Appleton, WI, 54913",54913,44.263142,-88.470413
4960,4961,Taco Bell (3221 East Express Court),4.3,78.0,"Burritos, Fast Food, Mexican",$,"3221 East Express Court, Appleton, WI, 54915",54915,44.258868,-88.359474
4961,4962,Subway (833 W Wisconsin Ave),4.9,10.0,"Fast Food, Sandwich, American",$,"833 W Wisconsin Ave, Appleton, WI, 54914",54914,44.272660,-88.418680


In [15]:
# expand full_address column out to street, city, and state
restaurant_df[['street', 'city', 'state', 'zip']] = restaurant_df['full_address'].str.split(',', 3, expand=True)

In [16]:
# delete zip column
del restaurant_df['zip']

# reorder columns
restaurant_df = restaurant_df.reindex(columns=['id','name', 'score','ratings','category', 'price_range',
                                              'full_address', 'street', 'city', 'state', 'zip_code', 'lat', 'lng'])
restaurant_df

,id,name,score,ratings,category,price_range,full_address,street,city,state,zip_code,lat,lng
6,7,Jinsei Sushi,4.7,63.0,"Sushi, Asian, Japanese",$,"1830 29th Ave S, Birmingham, AL, 35209",1830 29th Ave S,Birmingham,AL,35209,33.480440,-86.790440
13,14,Panera (521 Fieldstown Road),4.6,44.0,"Breakfast and Brunch, salad, Sandwich, Family ...",$,"521 Fieldstown Road, Gardendale, AL, 35071",521 Fieldstown Road,Gardendale,AL,35071,33.651407,-86.819247
15,16,Jeni's Splendid Ice Cream (Pepper Place),5.0,20.0,"Ice Cream &amp; Frozen Yogurt, Comfort Food, D...",$$$,"219 29th St S, Birmingham, AL, 35233",219 29th St S,Birmingham,AL,35233,33.516600,-86.789950
18,19,Falafel Cafe,4.9,48.0,"Middle Eastern, Mediterranean, Vegetarian, Gre...",$,"401 19th St S, Birmingham, AL, 35233",401 19th St S,Birmingham,AL,35233,33.508353,-86.803170
19,20,MrBeast Burger (838 Odum Road),3.7,19.0,"American, Burgers, Sandwich",$$,"838 Odum Road, Gardendale, AL, 35071",838 Odum Road,Gardendale,AL,35071,33.645480,-86.826260
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4951,4952,Toppers Pizza (Menomonee Falls),4.2,49.0,Pizza,$,"N88W15515 Main St., Menomonie Falls, WI, 53051",N88W15515 Main St.,Menomonie Falls,WI,53051,43.178895,-88.102625
4957,4958,Jersey Mike’s Subs (149 North Mall Drive),4.9,38.0,"Sandwich, American",$,"149 North Mall Drive, Appleton, WI, 54913",149 North Mall Drive,Appleton,WI,54913,44.263142,-88.470413
4960,4961,Taco Bell (3221 East Express Court),4.3,78.0,"Burritos, Fast Food, Mexican",$,"3221 East Express Court, Appleton, WI, 54915",3221 East Express Court,Appleton,WI,54915,44.258868,-88.359474
4961,4962,Subway (833 W Wisconsin Ave),4.9,10.0,"Fast Food, Sandwich, American",$,"833 W Wisconsin Ave, Appleton, WI, 54914",833 W Wisconsin Ave,Appleton,WI,54914,44.272660,-88.418680


In [17]:
# set index to id
restaurant_df.set_index("id", inplace=True)
restaurant_df.head()

,name,score,ratings,category,price_range,full_address,street,city,state,zip_code,lat,lng
id,,,,,,,,,,,,
7,Jinsei Sushi,4.7,63.0,"Sushi, Asian, Japanese",$,"1830 29th Ave S, Birmingham, AL, 35209",1830 29th Ave S,Birmingham,AL,35209,33.480440,-86.790440
14,Panera (521 Fieldstown Road),4.6,44.0,"Breakfast and Brunch, salad, Sandwich, Family ...",$,"521 Fieldstown Road, Gardendale, AL, 35071",521 Fieldstown Road,Gardendale,AL,35071,33.651407,-86.819247
16,Jeni's Splendid Ice Cream (Pepper Place),5.0,20.0,"Ice Cream &amp; Frozen Yogurt, Comfort Food, D...",$$$,"219 29th St S, Birmingham, AL, 35233",219 29th St S,Birmingham,AL,35233,33.516600,-86.789950
19,Falafel Cafe,4.9,48.0,"Middle Eastern, Mediterranean, Vegetarian, Gre...",$,"401 19th St S, Birmingham, AL, 35233",401 19th St S,Birmingham,AL,35233,33.508353,-86.803170
20,MrBeast Burger (838 Odum Road),3.7,19.0,"American, Burgers, Sandwich",$$,"838 Odum Road, Gardendale, AL, 35071",838 Odum Road,Gardendale,AL,35071,33.645480,-86.826260


In [18]:
# number of rows in cleaned restaurant_df
len(restaurant_df)

1907

In [24]:
perc_changed = len(restaurant_df)/len(original_restaurant_df) * 100
print(f'Now working with {perc_changed}% of original data.')

Now working with 38.14% of original data.


In [11]:
# check for missing values in menu_data
# first determine complete number of rows
# 403,240 rows 
menu_data.isnull()

# next determine which rows any missing values are in
menu_data.isnull().sum()

# all null values are in 'description' column 

# last determine if these are necessary to remove or if removing could cause us to loose some usable data
# 86,285 rows / 403,240 rows

restaurant_id        0
category             0
name                 0
description      86285
price                0
dtype: int64

In [35]:
# how much of the data would be lost if all rows with null values were deleted
perc_data_null = 86285/403240 * 100
print(round(perc_data_null))

21


In [ ]:
# delete the whole column or null rows? or keep all?

# re-inspect menu_data in excel to determine if 'description' column is valuable

# determined it IS useful data in certain areas and worth keeping the 'description' column
# for example for 'name' column "combination plate" is listed but in the 'description' column it tells us these for a 
# particular restaurant are "buffalo wings"

# instances where it is not useful, for example where a soda is listed in 'name' like coke or sprite, the 'description' 
# is left blank, these we can ignore so we will keep the entire column of 'description'

# instances where it would be counter productive to delete the entire row for a null 'description' are where 'name' gives us
# sides like 'fries' or 'okra' and 'description' is blank, if we delete the entire row, we would lose the information of 
# what sides certain restaurants offer

# final resolution is to keep all rows and keep 'description' column

#### Create database connection

In [47]:
# create restaurant_db in pgAdmin, then run database connection code
connection_string = "enterprisedb:password@localhost:5444/restaurant_db"
engine = create_engine(f'postgresql://{connection_string}')

In [48]:
# confirm tables
engine.table_names()

[]

#### Load DataFrames into database

In [49]:
original_restaurant_df.to_sql(name='original_restaurants', con=engine, if_exists='append', index=True)

In [50]:
restaurant_df.to_sql(name='restaurants', con=engine, if_exists='append', index=True)

In [51]:
menu_data.to_sql(name='menu', con=engine, if_exists='append', index=True)